# Atividade 2

```{admonition} Prazo

A lista é individual e deverá ser entregue até o dia 29/11/21, em editor de texto. Os trabalhos deverão ser entregues em ary.sabbag@ufpr.br
```

## **I-** Considere que se deseja estabelecer a tarifa de um plano de seguros que cobre consultas médicas. O número de consultas médicas de cada segurado por ano segue uma distribuição de Poisson com parâmetro 3 e o custo de cada consulta é igual a R$ 100,00. O plano de seguro está sendo prospectado para uma carteira com 10.000 segurados. No estabelecimento do prêmio tem-se como condição uma probabilidade de ruína igual a 0,025 e os carregamentos comerciais dados por:

Despesa administrativa – 10%;

Comissão de corretagem - 20%

Margem de lucro – 10%.

Com base nestas informações resolva as questões abaixo:

In [28]:
# Dados retirados do enunciado
parametro_poisson = 3
custo_consulta = 100
numero_segurados = 10000
prob_ruina = .025
despesas_administrativas = .1
comissao_corretagem = .2
margem_lucro = .1

### 1. Determine o número esperado de consultas por ano na carteira

O número de consultas $Y$ por ano na carteira é dado pela soma das consultas dos segurados $X_{i}$:

$$
Y = \sum^{N}_{i=1} X_{i}
$$

Como $X_{i}$ segue uma Poisson($\theta$), Y segue uma Poisson($N\theta$). Logo, $E[Y] = N\theta$:

In [29]:
from IPython.display import display, Latex

esperanca_Y = numero_segurados * parametro_poisson

display(Latex(f'E[Y] = {esperanca_Y}'))

<IPython.core.display.Latex object>

### 2. Qual a probabilidade de um segurado ter nenhuma consulta no período de 1 ano

A probabilidade de um segurado não ter nenhuma consulta é dado por $P[X=0]$ onde X segue uma distribuição Poisson(3)

In [30]:
from scipy import stats

prob_segurado_zero_consultas = stats.poisson.pmf(0, parametro_poisson)

display(Latex(f'P[X=0] = {prob_segurado_zero_consultas}'))

<IPython.core.display.Latex object>

### 3. Qual o número esperado de segurados com nenhuma consulta no período de 1 ano

Como mostrado no ítem 1, o número esperado de consultas é o resultado do produto entre o número de segurados (N) e o valor esperado de $X_{i}$. Utilizando a probabilidade de P[X=0] do ítem 2, temos que o número esperado de segurados com nenhuma consulta é dado $N*P[X=0]$:

In [31]:
numero_segurados_zero_consultas = numero_segurados * prob_segurado_zero_consultas

display(Latex(f'Y_{{X_{{i}} = 0}} = {numero_segurados_zero_consultas}'))

<IPython.core.display.Latex object>

### 4. Qual o prêmio de risco da carteira?

O prêmio de risco P da carteira é dado por:

$$
P = E[cX_{1} + cX_{2} + cX_{3} ... cX_{N}] = cE[Y] = cN\theta
$$

In [32]:
premio_de_risco = custo_consulta * numero_segurados * parametro_poisson

display(Latex(f'P = R\$ {premio_de_risco}'))

<IPython.core.display.Latex object>

### 5. Considerando a probabilidade de ruína adotada qual o prêmio de puro da carteira, dado que não se use capital de garantia?

O prêmio puro PP da carteira é dado por:

$$
PP = (1 + \lambda)P
$$

O prêmio de risco P é conhecido (ítem 4) e a carga de segurança $\lambda$ pode ser encontrada uma vez que sabemos probabilidade de ruína adotada e definido capital de garantia igual a zero.

Utilizando o Teorema Central do Limite, temos:

$$
\lambda = \frac{-\frac{U_{o}}{c} + Z_{\epsilon}\sqrt{N\theta}}{N\theta}
$$

In [33]:
import numpy as np

Z_prob_ruina = stats.norm.ppf(1 - prob_ruina)
capital_garantia = 0
n_theta = numero_segurados * parametro_poisson

carga_seguranca = (-capital_garantia/custo_consulta + Z_prob_ruina*(np.sqrt(n_theta)) )/ n_theta

display(Latex(f'\lambda = {carga_seguranca}'))

<IPython.core.display.Latex object>

Com a carga de segurança definida, podemos calcular o prêmio puro PP:

In [34]:
premio_puro = (1 + carga_seguranca)*premio_de_risco

display(Latex(f'PP = R\$ {premio_puro:.2f}'))

<IPython.core.display.Latex object>

### 6. Determine o prêmio comercial individual mensal

Seja o prêmio comercial PC dado por:

$$
PC = \frac{PP}{1-(DA + CC + L)}
$$

Temos que o prêmio comercial individual $PC_{i}$ é dado por:

$$
PC_{i} = PC/N
$$

Como os prêmios até aqui foram calculados para o período de 1 ano, o prêmio comercial individual mensal $PC_{i_{mensal}}$ é dado por:

$$
PC_{i_{mensal}} = PC_{i}/12
$$

In [35]:
premio_comercial = premio_puro / (1 - (despesas_administrativas + comissao_corretagem + margem_lucro))

premio_comercial_individual = premio_comercial / numero_segurados

premio_comercial_individual_mensal = premio_comercial_individual / 12

display(Latex(f'PC_{{i_{{mensal}}}} = R\$ {premio_comercial_individual_mensal:.2f}'))

<IPython.core.display.Latex object>

### 7. Mantendo a probabilidade de ruína adotada, se for cobrado um prêmio comercial individual mensal igual a R$ 38,00 qual deve ser o capital de garantia a ser adotado?

Revertendo os passos dos ítens anteriores, partindo do $PC_{i_{mensal}}$ do ítem 6 até a equação da carga de segurança $\lambda$ do ítem 5, temos:

$$
PC_{i} = 12PC_{i_{mensal}} \\
PC = NPC_{i} \\
PP = (1 - (DA + CC + L))PC \\
\lambda = \frac{PP - P}{P} \\
U_{o} = c(Z_{\epsilon}\sqrt{N\theta} - \lambda N\theta)
$$

In [36]:
premio_comercial_individual_mensal = 38

premio_comercial_individual = premio_comercial_individual_mensal * 12

premio_comercial = premio_comercial_individual * numero_segurados

premio_puro = (1 - (despesas_administrativas + comissao_corretagem + margem_lucro)) * premio_comercial

carga_seguranca = (premio_puro - premio_de_risco)/premio_de_risco

capital_garantia = custo_consulta * (Z_prob_ruina * np.sqrt(n_theta) - carga_seguranca * n_theta)

display(Latex(f'U_{{o}} = R\$ {capital_garantia:.2f}'))

<IPython.core.display.Latex object>

### 8. Se for adotado um capital de garantia igual a R\$ 250.000,00 e se for cobrado um prêmio comercial individual mensal igual a R\$ 38,00 qual será a probabilidade de ruína que a seguradora estará incorrendo?

Utilizando o resultado do ítem anterior que também teve como base um $PC_{i_{mensal}} = R\$ 38,00$ e aplicando o capital de garantia de R\$ 250.000,00, descobrimos o valor de $Z_{\epsilon}$:

$$
Z_{\epsilon} = \frac{\frac{U_{o}}{c} + \lambda N\theta}{\sqrt{N\theta}}
$$

In [37]:
capital_garantia = 250000

Z_prob_ruina = abs(((capital_garantia/custo_consulta) + carga_seguranca*n_theta)/np.sqrt(n_theta))

display(Latex(f'Z_{{\epsilon}} = {Z_prob_ruina}'))

<IPython.core.display.Latex object>

Econtrado o $Z_{\epsilon}$, podemos consultar qual a probabilidade de $P[Z \gt z_{\epsilon}]$ correspondente:

In [38]:
prob_ruina = 1 - stats.norm.cdf(Z_prob_ruina)

display(Latex(f'\epsilon = {prob_ruina}'))

<IPython.core.display.Latex object>

### 9. Se for adotado um capital de garantia igual a R\$ 250.000,00 e se for cobrado um prêmio comercial individual mensal igual a R\$ 38,00 qual deverá ser a comissão de corretagem a ser adotada se for desejada a probabilidade de ruína de 0,025?

Para encontrar a comissão de corretagem CC, devemos primeiramente recalcular a carga de segurança a fim de encontrar o novo prêmio puro que por sua vez, nos permitirá calcular o novo prêmio comercial:

$$
\lambda = \frac{-\frac{U_{o}}{c} + Z_{\epsilon}\sqrt{N\theta}}{N\theta}
$$

In [39]:
prob_ruina = .025
capital_garantia = 250000

Z_prob_ruina = stats.norm.ppf(1 - prob_ruina)
n_theta = numero_segurados * parametro_poisson

carga_seguranca = (-capital_garantia/custo_consulta + Z_prob_ruina*(np.sqrt(n_theta)) )/ n_theta

display(Latex(f'\lambda = {carga_seguranca}'))

<IPython.core.display.Latex object>

Aplicando a carga de segurança, temos:

In [40]:
premio_puro = (1 + carga_seguranca)*premio_de_risco

display(Latex(f'PP = R\$ {premio_puro}'))

<IPython.core.display.Latex object>

Já o prêmio comercial $PC$ para um $PC_{i_{mensal}} = R\$38,00$ temos do ítem 7:

In [41]:
premio_comercial_individual_mensal = 38
premio_comercial_individual = premio_comercial_individual_mensal * 12
premio_comercial = premio_comercial_individual * numero_segurados

display(Latex(f'PC = R\$ {premio_comercial}'))

<IPython.core.display.Latex object>

Logo, isolando a comissão de corretagem na fórmula do PC, temos:

$$
CC = 1 - DA - L - \frac{PP}{PC}
$$

In [42]:
comissao_corretagem = 1 - despesas_administrativas - margem_lucro - premio_puro/premio_comercial

display(Latex(f'CC = {comissao_corretagem:.4f}'))

<IPython.core.display.Latex object>

## **II-** Considere uma carteira composta por 4000 segurados, onde cada um tem um veículo cujo valor corresponde a R$ 20.000,00. O contrato feito entre o segurado e a seguradora declara que só é coberto o evento de perda total. Sabe-se que a probabilidade de um veículo ter sinistro de perda total no período de um ano é igual a 0,02. A seguradora admite uma probabilidade de ruína igual a 0,05 e adota como carregamento para despesas administrativas, comissão de corretagem e margem de lucro, respectivamente os percentuais de 15%, 20% e 10%. O período do contrato de seguro é de um ano. Com base nestas informações responda as questões abaixo:

In [43]:
numero_segurados = 4000
custo_perda_total = 20000
prob_sinistro = .02
prob_ruina = .05
despesas_administrativas = .15
comissao_corretagem = .20
margem_lucro = .1

### a. Qual o prêmio de risco (P) da carteira no período de um ano?

A probabilidade de sinistro de cada segurado $X_{i}$ segue uma Bernoulli(p), assim a soma das $N$ probabilidades individuais de seguro $Y$ segue uma Binomial(N, p), logo: 

$$
P = E[cX_{1} + cX_{2} + ... + cX_{n}] = cE[Y] = cNp
$$

In [44]:
premio_de_risco = custo_perda_total * numero_segurados * prob_sinistro

display(Latex(f'P = R\$ {premio_de_risco:.2f}'))

<IPython.core.display.Latex object>

### b. Com a probabilidade de ruína adotada qual o prêmio puro a ser praticado (PP)?

Assim como na questão 1 ítem 5, o prêmio puro PP da carteira é dado por:

$$
PP = (1 + \lambda)P
$$

O prêmio de risco P é conhecido do ítem anterior e a carga de segurança $\lambda$ pode ser encontrada uma vez que sabemos probabilidade de ruína adotada. Como o capital de garantia não foi revelado até aqui, segue que $U_{o} = 0$

Utilizando o Teorema Central do Limite para isolar $\lambda$ no cálculo da probabilidade de ruína $P[U_{t} \lt 0] = \epsilon$, temos:

$$
\lambda = \frac{-\frac{U_{o}}{c} + Z_{\epsilon}\sqrt{Np(1-p)}}{Np}
$$

In [45]:
Z_prob_ruina = stats.norm.ppf(1 - prob_ruina)
capital_garantia = 0
n_p = numero_segurados * prob_sinistro

carga_seguranca = (-capital_garantia/custo_perda_total + Z_prob_ruina*(np.sqrt(n_p*(1-prob_sinistro))) )/ n_p

display(Latex(f'\lambda = {carga_seguranca}'))

<IPython.core.display.Latex object>

Logo, calculando o PP temos:

In [46]:
premio_puro = (1 + carga_seguranca) * premio_de_risco

display(Latex(f'P = R\$ {premio_puro:.2f}'))

<IPython.core.display.Latex object>

### c. Se for adotado um carregamento de segurança igual a 0,07 (7%) qual o prêmio puro praticado e qual a probabilidade de ruína associada a esta prática?

Seja o prêmio puro PP dado por:

$$
PP = (1 + \lambda)P
$$

Temos:

In [47]:
carga_seguranca = .07
premio_puro = (1 + carga_seguranca) * premio_de_risco

display(Latex(f'P = R\$ {premio_puro:.2f}'))

<IPython.core.display.Latex object>

Isolando $Z_{\epsilon}$, temos:

$$
Z_{\epsilon} = \frac{\frac{U_{o}}{c} + \lambda Np}{\sqrt{Np(1-p)}}
$$

In [48]:
Z_prob_ruina = abs(((capital_garantia/custo_perda_total) + carga_seguranca*n_p)/np.sqrt(n_p*(1-prob_sinistro)))

display(Latex(f'Z_{{\epsilon}} = {Z_prob_ruina}'))

<IPython.core.display.Latex object>

Econtrado o $Z_{\epsilon}$, podemos consultar qual a probabilidade de $P[Z \gt z_{\epsilon}]$ correspondente:

In [49]:
prob_ruina = 1 - stats.norm.cdf(Z_prob_ruina)

display(Latex(f'\epsilon = {prob_ruina}'))

<IPython.core.display.Latex object>

### d. Com a prática do carregamento de segurança de 7% qual deverá ser o capital de garantia ($U_{0}$) a ser adotado de forma que se mantenha a probabilidade de ruína igual a 0,05?

$$
U_{0} = c(Z_{\epsilon}\sqrt{Np(1-p)} - \lambda Np)
$$

In [50]:
prob_ruina = .05
Z_prob_ruina = stats.norm.ppf(1 - prob_ruina)

capital_garantia = custo_perda_total * (Z_prob_ruina * np.sqrt(n_p*(1-prob_sinistro)) - carga_seguranca * n_p)

display(Latex(f'U_{{0}} = R\$ {capital_garantia:.2f}'))

<IPython.core.display.Latex object>

### e. Com as condições definidas anteriormente, qual o prêmio comercial (PC) praticado na carteira como um todo?

Com as condições definidas anteriormente (carga de segurança = 7%), temos que o prêmio comercial PC é dado por:

$$
PC = \frac{PP}{1-(DA + CC + L)}
$$

Temos que o prêmio comercial individual $PC_{i}$ é dado por:

$$
PC_{i} = PC/N
$$

Como os prêmios até aqui foram calculados para o período de 1 ano, o prêmio comercial individual mensal $PC_{i_{mensal}}$ é dado por:

$$
PC_{i_{mensal}} = PC_{i}/12
$$

In [51]:
carga_seguranca = .07
premio_puro = (1 + carga_seguranca) * premio_de_risco

premio_comercial = premio_puro / (1 - (despesas_administrativas + comissao_corretagem + margem_lucro))

display(Latex(f'PC = R\$ {premio_comercial:.2f}'))

<IPython.core.display.Latex object>

### f. Adotando-se um parcelamento em 12 vezes sem juros, qual será o prêmio comercial individual por parcela?

Inicialmente, calculamos o prêmio comercial individual $PC_{i}$:

$$
PC_{i} = PC/N
$$

Assim, temos que em um cenário de parcelamento em 12 vezes, o valor da parcela será dado por:

$$
PC_{i_{parcela}} = PC_{i}/12
$$

In [52]:
premio_comercial_individual = premio_comercial / numero_segurados

premio_comercial_individual_parcela = premio_comercial_individual / 12

display(Latex(f'PC_{{i_{{parcela}}}} = R\$ {premio_comercial_individual_parcela:.2f}'))

<IPython.core.display.Latex object>

### g. Se a área comercial desejar praticar um prêmio individual por parcela igual a R\$ 60,00, em vez de um carregamento de margem de lucro igual a 10%, qual deverá ser o carregamento de margem de lucro a ser adotado?

Definindo o prêmio comercial individual por parcela em R\$ 60,00, calculamos o prêmio comercial como:

$$
PC = N(12PC_{i_{parcela}})
$$

In [53]:
premio_comercial_individual_parcela = 60
premio_comercial = numero_segurados * 12 * premio_comercial_individual_parcela

display(Latex(f'PC = R\$ {premio_comercial:.2f}'))

<IPython.core.display.Latex object>

Para manter a equação equilibrada sem alterar o prêmio puro, isolamos a margem de lucro L da equação do ítem E:

$$
L = 1 - DA - CC - \frac{PP}{PC}
$$

In [54]:
margem_lucro = 1 - despesas_administrativas - comissao_corretagem - premio_puro/premio_comercial

display(Latex(f'L = {margem_lucro:.4f}'))

<IPython.core.display.Latex object>

### h. Para qual tamanho de carteira poderíamos manter o prêmio comercial individual por parcela igual a R$ 60,00, adotando uma probabilidade de ruína igual a 0,05, uma margem de lucro igual a 10%, uma comissão de corretagem igual a 20% e uma despesa administrativa igual a 15%, sem a adoção de capital de garantia?

Seja o prêmio de risco $P = cNp$ e o prêmio puro $PP = (1 + \lambda)P = (1 + \lambda)cNp$, temos o prêmio comercial PC dado por:

$$
PC = \frac{(1 + \lambda) cNp}{1- (DA + CC + L)}
$$

Assim, utilizando o PC calculado do ítem G e os valores para DA + CC + L, além dos já conhecidos custo e probabilidade de sinistro, podemos isolar o número de segurados N:

$$
N = \frac{PC(1 - (DA + CC + L))}{(1 + \lambda)cp}
$$

In [57]:
margem_lucro = .1
comissao_corretagem = .2
despesas_administrativas = .15
carga_seguranca = .07

numero_segurados = premio_comercial * (1 - (despesas_administrativas + comissao_corretagem + margem_lucro)) / ((1 + carga_seguranca) * custo_perda_total * prob_sinistro)
numero_segurados = round(numero_segurados)

display(Latex(f'N = {numero_segurados}'))

<IPython.core.display.Latex object>

### i. Com o capital de garantia adotado na questão d) qual será a probabilidade de lucro com a subscrição? 

Para que a carteira tenha lucro, é preciso encontrar o valor do PC sem a variável que representa o lucro seja maior que 0:

$$
L = 1 - DA - CC - \frac{PP}{PC} \rightarrow 1 - DA - CC - \frac{PP}{PC} \gt 0
$$